In [1]:
import requests, json
from datetime import datetime
slug = "maduro-in-us-custody-by-january-31"

url = f"https://gamma-api.polymarket.com/markets/slug/{slug}"

response = requests.get(url)

print(json.dumps(json.loads(response.text), indent=4))

{
    "id": "1094682",
    "question": "Maduro in U.S. custody by January 31?",
    "conditionId": "0xbfa45527ec959aacc36f7c312bd4f328171a7681ef1aeb3a7e34db5fb47d3f1d",
    "slug": "maduro-in-us-custody-by-january-31",
    "resolutionSource": "",
    "endDate": "2026-01-31T00:00:00Z",
    "startDate": "2026-01-03T07:49:39.785553Z",
    "image": "https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg",
    "icon": "https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg",
    "description": "This market will resolve to \u201cYes\u201d if Venezuelan President Nicolas Maduro is taken into custody by the U.S. government at any point by January 31, 2026, 11:59 PM ET. Otherwise, this market will resolve to \u201cNo\u201d.\n\nMaduro will be considered taken into U.S. government custody if U.S. government personnel (including military, CIA, personnel of another agency, or contractors acting under

In [2]:
clob_token_id= "7023463941941580393623777508894165086142929841541805476418845616988817847686"
start_time = datetime.datetime(2026,1,1,0,0).timestamp()
end_time = datetime.datetime(2026,1,3,0,0).timestamp()

print(start_time)
print(end_time)


url = f"https://clob.polymarket.com/prices-history?market={clob_token_id}&startTs={start_time}&endTs={end_time}&fidelity=&3600interval=max"

response = requests.get(url)
print(json.dumps(json.loads(response.text), indent=4))

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

## Markets

In [ ]:
def date_to_unix(date):
    '''
    Takes in a date in mm/dd/yyyy format and returns the appropriate unix date
    '''
    date_dt = datetime.strptime(date,"%m/%d/%Y")
    date_unix = int(date_dt.timestamp())
    return(date_unix)

In [91]:
jan_first_unix = date_to_unix("01/01/2026")
jan_second_unix = date_to_unix("01/02/2026")
jan_third_unix = date_to_unix("01/03/2026")
jan_thirty_unix = date_to_unix("01/30/2026")
jan_thirtyone_unix = date_to_unix("01/31/2026")

In [102]:
#playing around with the query parameters to get some info on what we can manipulate to get what we want
closed_markets_url = f"https://gamma-api.polymarket.com/markets?order=id&ascending=false&closed=true&limit=200&end_date_min={jan_thirty_unix}&end_date_max={jan_thirtyone_unix}&volume_num_min=11000000&volume_num_max=11100000"

#now we need to get start and end date easily: this year?

closed_response = requests.get(closed_markets_url)
markets_json = closed_response.json()



In [103]:
markets_json

[{'id': '1094682',
  'question': 'Maduro in U.S. custody by January 31?',
  'conditionId': '0xbfa45527ec959aacc36f7c312bd4f328171a7681ef1aeb3a7e34db5fb47d3f1d',
  'slug': 'maduro-in-us-custody-by-january-31',
  'resolutionSource': '',
  'endDate': '2026-01-31T00:00:00Z',
  'startDate': '2026-01-03T07:49:39.785553Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg',
  'description': 'This market will resolve to “Yes” if Venezuelan President Nicolas Maduro is taken into custody by the U.S. government at any point by January 31, 2026, 11:59 PM ET. Otherwise, this market will resolve to “No”.\n\nMaduro will be considered taken into U.S. government custody if U.S. government personnel (including military, CIA, personnel of another agency, or contractors acting under direct U.S. government authority) physica

Now we need a way to easily get the valuable info from the market json

In [115]:
def get_clob(market):
    '''
    Takes in the market json and returns the two clob ids (idk why there are 2)
    '''
    clob_list = json.loads(market[0]['clobTokenIds'])
    return clob_list

maduro_clob = get_clob(markets_json)[0]
maduro_clob

'7023463941941580393623777508894165086142929841541805476418845616988817847686'

## Clob 

This code has the right structure to use the clob tokens (note the indexing and split stuff you have to do) to get history, but it seems like these markets might be too old to have clob history available. 

Next step with this code is to make it into a function

In [4]:
#some interesting slugs to investigate: 
vaccine_2020slug = "will-there-be-an-emergency-use-authorization-granted-for-a-covid-19-vaccine-prior-to-nov-3"
drake_2020slug = "will-drake-release-a-new-studio-album-before-the-end-of-2020"
vaccine_url = f"https://gamma-api.polymarket.com/markets/slug/{vaccine_2020slug}"
drake_url = f"https://gamma-api.polymarket.com/markets/slug/{drake_2020slug}"

In [5]:
response_vaccine_raw = requests.get(vaccine_url)
response_vaccine = response_vaccine_raw.json()
vaccine_clob_ids = response_vaccine['clobTokenIds']
vaccine_created = response_vaccine['createdAt']
vaccine_closed = response_vaccine['closedTime']
vaccine_volume = response_vaccine['volume']

In [6]:
vaccine_closed

'2020-11-02 16:33:23+00'

In [7]:
response_drake_raw = requests.get(drake_url)
response_drake = response_drake_raw.json()
drake_clob_ids = response_drake['clobTokenIds']
drake_created = response_drake['createdAt']
drake_closed = response_drake['closedTime']
drake_voliume = response_drake['volume']

In [14]:

vaccine_created_utc = vaccine_created[:-1] + '+00:00'
vaccine_created_dt = datetime.fromisoformat(vaccine_created_utc)
vaccine_created_unix = vaccine_created_dt.timestamp()
vaccine_closed_utc = vaccine_closed + ':00'
vaccine_closed_dt = datetime.fromisoformat(vaccine_closed_utc)
vaccine_closed_unix = vaccine_closed_dt.timestamp()

In [43]:
test_clob = vaccine_clob_ids.split(",")[1][2:-2]
test_clob

'76498560192118683650733506412944777208149122792035467707472962601509300857635'

In [ ]:
vaccine_history_url = f"https://clob.polymarket.com/prices-history?market={test_clob}&interval=max"
print(vaccine_history_url)
vaccine_history_response = requests.get(vaccine_history_url)
vaccine_history_json = vaccine_history_response.json()

https://clob.polymarket.com/prices-history?market=76498560192118683650733506412944777208149122792035467707472962601509300857635&interval=max
{
    "history": []
}


In [ ]:
#play with this a little more to see if we can actually get some of the market using the date parameters
maduro_url = f"https://clob.polymarket.com/prices-history?market={maduro_clob}&interval=max"
maduro_history = requests.get(maduro_url)
maduro_json = maduro_history.json()
maduro_json

{'history': []}